<a href="https://colab.research.google.com/github/malikkarim14/Failed-Orders-at-Gett/blob/main/Preprocessing_Gett_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Data Project/Insights from Failed Orders (Gett)/data_orders.csv")
data.head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,cancellations_time_in_seconds
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,4,1,198.0
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0
2,12:07:50,-0.969520,51.455544,477.0,3000582891479,4,1,46.0
3,13:50:20,-1.054671,51.460544,658.0,3000582941169,4,1,62.0
4,21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_datetime                 10716 non-null  object 
 1   origin_longitude               10716 non-null  float64
 2   origin_latitude                10716 non-null  float64
 3   m_order_eta                    2814 non-null   float64
 4   order_gk                       10716 non-null  int64  
 5   order_status_key               10716 non-null  int64  
 6   is_driver_assigned_key         10716 non-null  int64  
 7   cancellations_time_in_seconds  7307 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 669.9+ KB


To improve data quality slightly. The values in `order_status_key` and `is_driver_assigned_key` are not informative of the contents, but rather point to some internal description. We could replace the `1`s in `is_driver_assigned_key` with the string `Yes` and the `0`s with the string `No` to provide more information. The `order_status_key` column can be treated the same way. Furthermore, the column names appear a little technical, but we can change them.

In [ ]:
data["is_driver_assigned"] = np.where(data["is_driver_assigned_key"] == 1, "Yes", "No")
data["order_status"] = np.where(data["order_status_key"] == 4, "Client Cancelled", "System Reject")

data.drop(columns=["is_driver_assigned_key", "order_status_key"], inplace=True)

In [ ]:
data = data.rename(columns={"order_datetime": "order_time"})

We'll take the hour from `order_time` and make a new column for it because we're curious about when these failures occur and if there's a specific time of day when one category outnumbers others.

In [ ]:
# extract hour from the time column
data["order_hour"] = data["order_time"].str.split(":").apply(lambda split: split[0])

Following that, we'll remove some columns that won't be used in the analysis. Because there are too many missing values for the `m_order_eta` column, a new dataframe will be created.

In [ ]:
data_cleaned = data.drop(columns=['order_time','origin_longitude','origin_latitude','m_order_eta'])

In [ ]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_gk                       10716 non-null  int64  
 1   cancellations_time_in_seconds  7307 non-null   float64
 2   is_driver_assigned             10716 non-null  object 
 3   order_status                   10716 non-null  object 
 4   order_hour                     10716 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 418.7+ KB


Because there are 3409 rows with no value (NaN value) in the `cancellations_time_in_seconds`, we will impute using the average value based on the `order_hour` and `is_driver_assigned`.

In [ ]:
data_cleaned['cancellations_time_in_seconds'] = data_cleaned['cancellations_time_in_seconds'].fillna(data_cleaned.groupby(['order_hour','is_driver_assigned'])['cancellations_time_in_seconds'].transform('mean'))
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_gk                       10716 non-null  int64  
 1   cancellations_time_in_seconds  10716 non-null  float64
 2   is_driver_assigned             10716 non-null  object 
 3   order_status                   10716 non-null  object 
 4   order_hour                     10716 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 418.7+ KB


In [ ]:
# New dataframe for eta column
column = {'order_eta':data["m_order_eta"],'order_hour':data["order_hour"]}
eta = pd.DataFrame(data=column)
eta.head()

,order_eta,order_hour
0,60.0,18
1,NaN,20
2,477.0,12
3,658.0,13
4,NaN,21


In [ ]:
eta = eta.dropna()
eta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 10715
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_eta   2814 non-null   float64
 1   order_hour  2814 non-null   object 
dtypes: float64(1), object(1)
memory usage: 66.0+ KB


In [ ]:
eta.to_excel("/content/drive/MyDrive/Data Project/Insights from Failed Orders (Gett)/etaonly.xlsx")

In [ ]:
data_cleaned.to_excel("/content/drive/MyDrive/Data Project/Insights from Failed Orders (Gett)/dataset.xlsx")